In [4]:
import torch
import torchvision 
import pandas as pd
import numpy as np
import argparse
from PIL import Image
import os
from tqdm import tqdm
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from torch.utils import data
from torchvision import transforms, datasets, models
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import time
import utils_ObjectDetection as utils
from torch.utils.data import BatchSampler,SequentialSampler,RandomSampler
import torch.nn as nn
from torch.nn.parallel import DistributedDataParallel as DDP
import math 
import ITB_RECALL_PRECISION_F1 as rp_f1

In [5]:
def make_prediction(model, img, threshold):
    model.eval()
    preds = model(img)
    for id in range(len(preds)) :
        idx_list = []

        for idx, score in enumerate(preds[id]['scores']) :
            
            if score > threshold : 
                idx_list.append(idx)

        preds[id]['boxes'] = preds[id]['boxes'][idx_list]
        preds[id]['labels'] = preds[id]['labels'][idx_list]
        preds[id]['scores'] = preds[id]['scores'][idx_list]

    return preds

def collate_fn(batch):
    return tuple(zip(*batch))


def get_transform():
    custom_transforms = []
    custom_transforms.append(torchvision.transforms.ToTensor())
    return torchvision.transforms.Compose(custom_transforms)


def get_model_instance_segmentation(num_classes,train_layer):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True,trainable_backbone_layers=train_layer)
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

def plot_image_from_output(img, annotation):
    
    img = img.cpu().permute(1,2,0)
    
    fig,ax = plt.subplots(figsize=(20, 10))
    ax.imshow(img)
    
    for idx in range(len(annotation["boxes"])):
        xmin, ymin, xmax, ymax = annotation["boxes"][idx]

        if annotation['labels'][idx] == 1 :
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='b',facecolor='none')
        
        elif annotation['labels'][idx] == 2 :
            
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='g',facecolor='none')
            
        elif annotation['labels'][idx] == 3 :
        
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='orange',facecolor='none')
        else:
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='r',facecolor='none')

        ax.add_patch(rect)

    return plt


In [6]:

class simpleDataset(object):


    def __init__(self, dataset,resize,color, img_foler_path,transforms=None):
    #             self.root = root
            self.transforms = transforms
            self.adnoc = dataset
            self.ids = dataset.index
            self. filenames=dataset['path'].to_list()
            self.resize =resize
            self.color=color
            self.img_foler_path=img_foler_path
            
    def __getitem__(self, index):
        # Own coco file
        adnoc_df = self.adnoc
        # Image ID ,폴더에서 index번째 
        img_id = self.ids[index]

       # List: get annotation id from coco, index번째 annotation가져오기 [[1,2,3,4],[5,6,7,8]]
        annotation = adnoc_df['box'][img_id]


        # open the input image, 흑백=L , 단색=1
        img= Image.open(str(self.img_foler_path)+str(adnoc_df.loc[img_id]['path'])).convert(self.color)
        img= img.resize((int(img.width / self.resize), int(img.height / self.resize)))
      
        # number of objects in the image
        num_objs = len(annotation)

        # Bounding boxes for objects
        # In coco format, bbox = [xmin, ymin, width, height]
        # In pytorch, the input should be [xmin, ymin, xmax, ymax]
        boxes = []
        areas = []
        label = []
        for i in range(num_objs):

            xmin = annotation[i][0]
            ymin = annotation[i][1]
            xmax = xmin + annotation[i][2]
            ymax = ymin + annotation[i][3]
            l=annotation[i][4]
            area=annotation[i][2]*annotation[i][3]

            boxes.append([xmin/self.resize, ymin/self.resize, xmax/self.resize, ymax/self.resize])
          
            label.append(l)
            areas.append(area)

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        #areas= box size = width * height
        areas = torch.as_tensor(areas, dtype=torch.float32)

        # Labels (In my case, I only one class: target class or background)
        labels = torch.as_tensor(label, dtype=torch.int64)

        # Tensorise img_id
        img_id = torch.tensor([img_id])

        # Iscrowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        # Annotation is in dictionary format
        my_annotation = {}
        my_annotation["boxes"] = boxes
        my_annotation["labels"] = labels
        my_annotation["image_id"] = img_id
        my_annotation["area"] = areas
        my_annotation["iscrowd"] = iscrowd

        if self.transforms is not None:
            img = self.transforms(img)

        return img, my_annotation

    def __len__(self):
        return len(self.ids)



    # the total number of samples (optional)
    def __len__(self):
        return len(self.filenames)

    


In [7]:
class uncertainty_sampling:
    def __init__(self,model,confidence,batch,train_data_loader):
        self.model=model
        self.confidence=confidence
        self.batch=batch
        self.train_data_loader=train_data_loader


    def img_uncertainty(self,model,ascend):
        '''
        내림차순, 큰 수 부터
        ascend=False
        오름차순, 작은 수 부터
        ascend=True 
        
        '''
        img_id=[]
        pred_label=[]
        pred_scores=[]
        img_uncertianty=[]
        labels = []

        print('confidence=',self.confidence)
        for im, annot in tqdm(self.train_data_loader, position = 0, leave = True):
            im = list(img.to(device) for img in im)
            #test data 이미지 id 저장
            img_id.append(annot[0]['image_id'].item())

            for t in annot:
                labels += t['labels']

            with torch.no_grad():
                preds_adj = make_prediction(model, im, self.confidence)
                preds_adj = [{k: v.to(torch.device('cpu')) for k, v in t.items()} for t in preds_adj]

                #예측 label 저장
                pred_label.append(preds_adj[0]['labels'].tolist())

                #예측 score 저장
                pred_scores.append(preds_adj[0]['scores'].tolist())
                img_uncertianty.append(np.mean(preds_adj[0]['scores'].tolist()))

        model_pred=pd.DataFrame(img_id,columns=['img_id'])
        model_pred['label']=pred_label
        model_pred['score']=pred_scores
        model_pred['uncertatinty']=img_uncertianty


        model_pred.sort_values(by=['uncertatinty'],ascending=ascend,inplace=True)
        return model_pred




    def sampling(self,train,ascend):
    
        model_pred=self.img_uncertainty(model,ascend)
          
        if len(train)<self.batch:
            sample=train
            train_rest=[]
            train_rest_data_loader=[]
        else:
            #상위 10개 data 선택
            sample_idx=model_pred[:self.batch]['img_id'].tolist()
            sample=train.loc[sample_idx]
            train_rest=train.drop(sample.index)
            
            train_rest_dataset = simpleDataset(dataset=train_rest,
                                  resize=4,
                                  color='L',
                                  img_foler_path='D:/OBJ_PAPER/itb_p_500/',
                                  transforms=get_transform())

            train_rest_data_loader = torch.utils.data.DataLoader(train_rest_dataset,
                                                  batch_size=1,
                                                  shuffle=True,
                                                  num_workers=0,
                                                  collate_fn=collate_fn)

        sample_dataset = simpleDataset(dataset=sample,
                                  resize=4,
                                  color='L',
                                  img_foler_path='D:/OBJ_PAPER/itb_p_500/',
                                  transforms=get_transform())

        sample_data_loader = torch.utils.data.DataLoader(sample_dataset,
                                              batch_size=1,
                                              shuffle=True,
                                              num_workers=0,
                                              collate_fn=collate_fn)
        
      
        
        return sample,train_rest,train_rest_data_loader,sample_data_loader

In [8]:
def result_RPA(test_data_loader,model,param,iteration):
    
    model = get_model_instance_segmentation(4,5)
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)
    model.load_state_dict(torch.load(param))
    model.eval()


    confi=[0.5]
    RECALL=[]
    PRECISION=[]
    F1_SCORE=[]
    for c in confi: 
        labels = []
        preds_adj_all = []
        annot_all = []
        print('confidence=',c)
        for im, annot in tqdm(test_data_loader, position = 0, leave = True):
            im = list(img.to(device) for img in im)

        #     annot
            for t in annot:
                labels += t['labels']

            with torch.no_grad():
                preds_adj = make_prediction(model, im, c)
                preds_adj = [{k: v.to(torch.device('cpu')) for k, v in t.items()} for t in preds_adj]
                preds_adj_all.append(preds_adj)
                annot_all.append(annot)

        #precision=0.2
        iou_threds=0.5
        pr_f1=rp_f1.Precision_Recall_F1(iou_threds,preds_adj_all,annot_all)
      
        precision=pr_f1.PRECISION()
        recall=pr_f1.RECALL()
        f1_score=pr_f1.F1_SCORE()
      

        RECALL.append(recall)
        PRECISION.append(precision)
        F1_SCORE.append(f1_score)
    
    
    print(RECALL,'\n',PRECISION,'\n',F1_SCORE)
    result=pd.DataFrame(RECALL,columns=['recall_form','recall_table','recall_text'])
    result['precision_form']=PRECISION[0][0]
    result['precision_table']=PRECISION[0][1]
    result['precision_text']=PRECISION[0][2]

    result['f1_form']=F1_SCORE[0][0]
    result['f1_table']=F1_SCORE[0][1]
    result['f1_text']=F1_SCORE[0][2]
    
    
    return result

In [9]:
def random_sampling(train,batch):
    if len(train)<batch:
        sampled=train
        train_rest=[]
        
    else:
        sampled=train.sample(n=batch,random_state=42)
        train_rest=train.drop(sampled.index)
        
    sample_dataset = simpleDataset(dataset=sampled,
                                  resize=4,
                                  color='L',
                                  img_foler_path='D:/OBJ_PAPER/itb_p_500/',
                                  transforms=get_transform())
    
    sample_data_loader = torch.utils.data.DataLoader(sample_dataset,
                                              batch_size=1,
                                              shuffle=True,
                                              num_workers=0,
                                              collate_fn=collate_fn)
    return sampled,train_rest,sample_data_loader


In [12]:
def data_load_all(train,valid,test):
 
    # create own Dataset
    train_dataset = simpleDataset(dataset=train,
                                  resize=4,
                                  color='L',
                                  img_foler_path='D:/OBJ_PAPER/Data/itb_p_500/',
                                  transforms=get_transform())

    valid_dataset = simpleDataset(
                              dataset=valid,
                              resize=4,
                              color='L',
                              img_foler_path='D:/OBJ_PAPER/Data/itb_p_500/',
                              transforms=get_transform())

    test_dataset = simpleDataset(
                              dataset=test,
                              resize=4,
                              color='L',
                              img_foler_path='D:/OBJ_PAPER/Data/itb_p_500/',
                              transforms=get_transform())



    train_data_loader = torch.utils.data.DataLoader(train_dataset,
                                              batch_size=1,
                                              shuffle=False,
                                              num_workers=0,                                         
                                              collate_fn=collate_fn)

    valid_data_loader = torch.utils.data.DataLoader(valid_dataset,
                                              batch_size=1,
                                              shuffle=True,
                                              num_workers=0,
                                              collate_fn=collate_fn)

    test_data_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=1,
                                          shuffle=False,
                                          num_workers=0,
                                          collate_fn=collate_fn)

    return train_data_loader,valid_data_loader,test_data_loader

In [10]:
def data_load(train):
 
    # create own Dataset
    train_dataset = simpleDataset(dataset=train,
                                  resize=4,
                                  color='L',
                                  img_foler_path='D:/OBJ_PAPER/Data/itb_p_500/',
                                  transforms=get_transform())


    train_data_loader = torch.utils.data.DataLoader(train_dataset,
                                              batch_size=1,
                                              shuffle=False,
                                              num_workers=0,                                         
                                              collate_fn=collate_fn)

 

    return train_data_loader

In [21]:
data_path='D:/OBJ_PAPER/Data/3_fold_cv/'

train1=pd.read_pickle(data_path+'train1.pkl')
train2=pd.read_pickle(data_path+'train2.pkl')
train3=pd.read_pickle(data_path+'train3.pkl')

valid1=pd.read_pickle(data_path+'valid1.pkl')
valid2=pd.read_pickle(data_path+'valid2.pkl')
valid3=pd.read_pickle(data_path+'valid3.pkl')

train1_no_table=pd.read_pickle(data_path+'train1_no_table.pkl')
train2_no_table=pd.read_pickle(data_path+'train2_no_table.pkl')
train3_no_table=pd.read_pickle(data_path+'train3_no_table.pkl')

test=pd.read_pickle(data_path+'test.pkl')

# Transfer learning

- 추가학습 없이 필리핀 문서 예측하기
- test set만 예측 

In [22]:
train_data_loader,valid_data_loader,test_data_loader=data_load(train3,valid3,test)

In [17]:
num_classes = 4
train_layer=5
iteration=1
RESULT=pd.DataFrame()
train_param_path='D:/OBJ_PAPER/model_param/model_fasterrcnn_bach5_p10_50_param.pt'

model = get_model_instance_segmentation(num_classes,train_layer)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


# move model to the right device
model.to(device)
model.load_state_dict(torch.load('D:/OBJ_PAPER/model_param/model_fasterrcnn_bach5_p10_50_param.pt'))

# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.0001)


result_batch=result_RPA(test_data_loader,model,train_param_path,iteration)
RESULT=RESULT.append(result_batch,ignore_index=True)

    
RESULT.to_csv('D:/OBJ_PAPER/result/FRCN_TL_no_train_3.csv',index=False)   

confidence= 0.5


100%|███████████████████████████████████████████████████████| 100/100 [00:08<00:00, 11.50it/s]


precision [0.27273, 0.9403, 0.62963]
recall [0.11321, 0.96923, 0.74843]
[[0.11321, 0.96923, 0.74843]] 
 [[0.27273, 0.9403, 0.62963]] 
 [[0.16, 0.95455, 0.68391]]


In [10]:
num_classes = 4
train_layer=5
iteration=1
RESULT=pd.DataFrame()


model = get_model_instance_segmentation(4,5)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.0001)

model.eval()


confi=[0.5]
RECALL=[]
PRECISION=[]
F1_SCORE=[]
for c in confi: 
    labels = []
    preds_adj_all = []
    annot_all = []
    print('confidence=',c)
    for im, annot in tqdm(test_data_loader, position = 0, leave = True):
        im = list(img.to(device) for img in im)

    #     annot
        for t in annot:
            labels += t['labels']

        with torch.no_grad():
            preds_adj = make_prediction(model, im, c)
            preds_adj = [{k: v.to(torch.device('cpu')) for k, v in t.items()} for t in preds_adj]
            preds_adj_all.append(preds_adj)
            annot_all.append(annot)

    #precision=0.2
    iou_threds=0.5
    pr_f1=rp_f1.Precision_Recall_F1(iou_threds,preds_adj_all,annot_all)

    precision=pr_f1.PRECISION()
    recall=pr_f1.RECALL()
    f1_score=pr_f1.F1_SCORE()


    RECALL.append(recall)
    PRECISION.append(precision)
    F1_SCORE.append(f1_score)


print(RECALL,'\n',PRECISION,'\n',F1_SCORE)
result=pd.DataFrame(RECALL,columns=['recall_form','recall_table','recall_text'])
result['precision_form']=PRECISION[0][0]
result['precision_table']=PRECISION[0][1]
result['precision_text']=PRECISION[0][2]

result['f1_form']=F1_SCORE[0][0]
result['f1_table']=F1_SCORE[0][1]
result['f1_text']=F1_SCORE[0][2]
    

RESULT=RESULT.append(result,ignore_index=True)

    
RESULT.to_csv('D:/OBJ_PAPER/result/FRCN_can.csv',index=False)   

confidence= 0.5


100%|█████████████████████████████████████████████████████████| 98/98 [00:12<00:00,  8.01it/s]


precision [0.0, 0.0, 0.00923]
recall [0.0, 0.0, 0.03448]
[[0.0, 0.0, 0.03448]] 
 [[0.0, 0.0, 0.00923]] 
 [[0.0, 0.0, 0.01456]]


### S사로 학습한 모델에 신규 데이터 추가 학습
### train_layer=5, valid set = 50, test set 예측

In [24]:
def train_TL(save_param_path,result_path,train_data_loader,valid_data_loader,test_data_loader):
    # 2 classes; Only target class or background
    num_classes = 4
    num_epochs = 100
    train_layer=1
    patience=10

    model = get_model_instance_segmentation(num_classes,train_layer)
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    #모델 저장 
    train_param_path=save_param_path
    # move model to the right device
    model.to(device)
    model.load_state_dict(torch.load('D:/OBJ_PAPER/model_param/model_fasterrcnn_bach5_p10_50_param.pt'))

    # parameters
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.Adam(params, lr=0.0001)

    #평균 loss
    avg_train_loss=[]
    avg_valid_loss=[]

    iteration = 0    

    RESULT=pd.DataFrame()

    model.train()
    not_save_count=0
    for epoch in range(num_epochs):

        # 모델이 학습되는 동안 trainning loss를 track
        train_losses = []
        # 모델이 학습되는 동안 validation loss를 track
        valid_losses = []
        st = time.time()
        for imgs, annotations in train_data_loader:

            imgs = list(img.to(device) for img in imgs)
            annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
            loss_dict = model(imgs, annotations)
            #batch size=10, 10개 loss 각각 도출
            losses = sum(loss for loss in loss_dict.values())
            train_losses.append(losses.item())       
            optimizer.zero_grad()
            losses.backward()
            optimizer.step()

        #이미지 한장당 평균 loss
        avg_train_loss.append(np.mean(train_losses).round(5))

        #validation, early_stop, save weights
        with torch.no_grad():

            for im, annot in valid_data_loader:
                im = list(img.to(device) for img in im)
                annot = [{k: v.to(device) for k, v in t.items()} for t in annot]
                val_loss_dict = model(im, annot)
                val_losses = sum(val_loss for val_loss in val_loss_dict.values())
                valid_losses.append(val_losses.item())

            epoch_val_loss=np.mean(valid_losses).round(5)
            avg_valid_loss.append(epoch_val_loss)  

        fi = time.time()     
        print('epoch:',epoch,'train_loss: ',np.mean(train_losses).round(5),'validation loss : ',np.mean(valid_losses).round(5),'time',fi-st)


        min_val_loss=np.min(avg_valid_loss)
        if min_val_loss>=epoch_val_loss:

            torch.save(model.state_dict(),train_param_path)
            not_save_count=0
            print('epoch:',epoch,'save model')

        else:
            not_save_count+=1
            model.load_state_dict(torch.load(train_param_path))
            if not_save_count>=patience:
                print('no more training')
                break


    fi = time.time()     
    print('iteration:',iteration,'train_loss: ',np.mean(train_losses).round(5),'val_loss: ',min_val_loss.round(5),'time',fi-st)
    model.load_state_dict(torch.load(train_param_path))

    result_batch=result_RPA(test_data_loader,model,train_param_path,iteration)
    RESULT=RESULT.append(result_batch,ignore_index=True)
    RESULT.to_csv(result_path,index=False)
    return RESULT

In [26]:
save_param_path='D:/OBJ_PAPER/model_param/FRCN_TL_valid50_layer1_train1_no_table_param.pt'
result_path='D:/OBJ_PAPER/result/FRCN_TL_valid50_layer1_train1_no_table_result.csv'

train_data_loader,valid_data_loader,test_data_loader=data_load_all(train1_no_table,valid1,test)
RESULT=train_TL(save_param_path,result_path,train_data_loader,valid_data_loader,test_data_loader)
RESULT

epoch: 0 train_loss:  0.20101 validation loss :  0.25377 time 40.627991914749146
epoch: 0 save model
epoch: 1 train_loss:  0.13921 validation loss :  0.23564 time 40.462947845458984
epoch: 1 save model
epoch: 2 train_loss:  0.11229 validation loss :  0.25218 time 40.42556166648865
epoch: 3 train_loss:  0.1135 validation loss :  0.25877 time 40.427637338638306
epoch: 4 train_loss:  0.11607 validation loss :  0.26152 time 40.54937672615051
epoch: 5 train_loss:  0.11749 validation loss :  0.26858 time 40.53559494018555
epoch: 6 train_loss:  0.11459 validation loss :  0.28413 time 40.78890514373779
epoch: 7 train_loss:  0.11594 validation loss :  0.26541 time 40.65884351730347
epoch: 8 train_loss:  0.117 validation loss :  0.28456 time 40.584728479385376
epoch: 9 train_loss:  0.11885 validation loss :  0.2494 time 40.62306880950928
epoch: 10 train_loss:  0.12075 validation loss :  0.29592 time 40.61904692649841
epoch: 11 train_loss:  0.11788 validation loss :  0.30538 time 40.6295986175537

100%|███████████████████████████████████████████████████████| 100/100 [00:08<00:00, 11.87it/s]


precision [0.52128, 0.0, 0.55357]
recall [0.92453, 0.0, 0.99359]
[[0.92453, 0.0, 0.99359]] 
 [[0.52128, 0.0, 0.55357]] 
 [[0.66667, 0.0, 0.71101]]


,recall_form,recall_table,recall_text,precision_form,precision_table,precision_text,f1_form,f1_table,f1_text
0,0.92453,0.0,0.99359,0.52128,0.0,0.55357,0.66667,0.0,0.71101


In [27]:
save_param_path='D:/OBJ_PAPER/model_param/FRCN_TL_valid50_layer1_train2_no_table_param.pt'
result_path='D:/OBJ_PAPER/result/FRCN_TL_valid50_layer1_train2_no_table_result.csv'

train_data_loader,valid_data_loader,test_data_loader=data_load_all(train2_no_table,valid2,test)
RESULT=train_TL(save_param_path,result_path,train_data_loader,valid_data_loader,test_data_loader)
RESULT

epoch: 0 train_loss:  0.20192 validation loss :  0.23116 time 40.97726821899414
epoch: 0 save model
epoch: 1 train_loss:  0.14075 validation loss :  0.24177 time 41.156251668930054
epoch: 2 train_loss:  0.13638 validation loss :  0.2673 time 41.094810485839844
epoch: 3 train_loss:  0.13855 validation loss :  0.25438 time 41.064656496047974
epoch: 4 train_loss:  0.13997 validation loss :  0.29963 time 41.21307682991028
epoch: 5 train_loss:  0.14256 validation loss :  0.25573 time 41.138429403305054
epoch: 6 train_loss:  0.14507 validation loss :  0.28261 time 41.18891215324402
epoch: 7 train_loss:  0.14624 validation loss :  0.27771 time 41.27439332008362
epoch: 8 train_loss:  0.14193 validation loss :  0.27271 time 41.3082230091095
epoch: 9 train_loss:  0.14382 validation loss :  0.26435 time 41.60800838470459
epoch: 10 train_loss:  0.14777 validation loss :  0.28618 time 41.382527351379395
no more training
iteration: 0 train_loss:  0.14777 val_loss:  0.23116 time 41.523423194885254
co

100%|███████████████████████████████████████████████████████| 100/100 [00:08<00:00, 11.78it/s]


precision [0.52222, 0.0, 0.6383]
recall [0.88679, 0.0, 0.96154]
[[0.88679, 0.0, 0.96154]] 
 [[0.52222, 0.0, 0.6383]] 
 [[0.65734, 0.0, 0.76727]]


,recall_form,recall_table,recall_text,precision_form,precision_table,precision_text,f1_form,f1_table,f1_text
0,0.88679,0.0,0.96154,0.52222,0.0,0.6383,0.65734,0.0,0.76727


In [28]:
save_param_path='D:/OBJ_PAPER/model_param/FRCN_TL_valid50_layer1_train3_no_table_param.pt'
result_path='D:/OBJ_PAPER/result/FRCN_TL_valid50_layer1_train3_no_table_result.csv'

train_data_loader,valid_data_loader,test_data_loader=data_load_all(train3_no_table,valid3,test)
RESULT=train_TL(save_param_path,result_path,train_data_loader,valid_data_loader,test_data_loader)
RESULT

epoch: 0 train_loss:  0.19544 validation loss :  0.28476 time 40.925448179244995
epoch: 0 save model
epoch: 1 train_loss:  0.12515 validation loss :  0.37451 time 40.968366622924805
epoch: 2 train_loss:  0.12729 validation loss :  0.35135 time 41.27254247665405
epoch: 3 train_loss:  0.1256 validation loss :  0.39794 time 41.041789054870605
epoch: 4 train_loss:  0.13407 validation loss :  0.37598 time 40.95479655265808
epoch: 5 train_loss:  0.12797 validation loss :  0.35521 time 41.18832588195801
epoch: 6 train_loss:  0.13161 validation loss :  0.37056 time 40.964760065078735
epoch: 7 train_loss:  0.1311 validation loss :  0.38517 time 41.02539324760437
epoch: 8 train_loss:  0.13212 validation loss :  0.37682 time 41.01503014564514
epoch: 9 train_loss:  0.13429 validation loss :  0.4118 time 41.06053948402405
epoch: 10 train_loss:  0.13273 validation loss :  0.39545 time 40.991140604019165
no more training
iteration: 0 train_loss:  0.13273 val_loss:  0.28476 time 41.13447952270508
conf

100%|███████████████████████████████████████████████████████| 100/100 [00:08<00:00, 11.69it/s]


precision [0.39837, 0.0, 0.4416]
recall [0.92453, 0.0, 0.99359]
[[0.92453, 0.0, 0.99359]] 
 [[0.39837, 0.0, 0.4416]] 
 [[0.55681, 0.0, 0.61144]]


,recall_form,recall_table,recall_text,precision_form,precision_table,precision_text,f1_form,f1_table,f1_text
0,0.92453,0.0,0.99359,0.39837,0.0,0.4416,0.55681,0.0,0.61144
